In [ ]:
import json
import os
import random
import pandas as pd
import tensorflow_addons as tfa
import numpy as np
import PIL.Image
import tensorflow as tf

from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import ReLU,LeakyReLU,ELU

from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Nadam, SGD, Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import HeNormal

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.utils import array_to_img, img_to_array, load_img

In [ ]:
def load_img_to_array(path):
    return img_to_array(load_img(path))


def resize(image, size):
    return tf.image.resize(image, size)


def aggressive_cropping(image, copies, crop_window, resize_smallest_side, output_shape):
    global img, resized_copies, crops

    if isinstance(resize_smallest_side, int):
        img = resize(image, (resize_smallest_side, resize_smallest_side))

    if isinstance(resize_smallest_side, (list, tuple)):
        resized_copies = [tf.image.resize(image, (size, size)) for size in resize_smallest_side]

    if isinstance(crop_window, int):
        if isinstance(resize_smallest_side, int):
            crops = [tf.image.random_crop(img, crop_window) for _ in range(copies)]
        elif isinstance(resize_smallest_side, (list, tuple)):
            crops = [tf.image.random_crop(img_, crop_window) for _ in range(copies) for img_ in
                     resized_copies]

    elif isinstance(crop_window, (list, tuple)):
        if isinstance(resize_smallest_side, int):
            crops = [tf.image.random_crop(img, crop_window) for _ in range(copies)]
        elif isinstance(resize_smallest_side, (list, tuple)):
            crops = [tf.image.random_crop(img_, crop_window) for _ in range(copies) for img_ in resized_copies]

    return [resize(crop_img, output_shape) for crop_img in crops]

In [ ]:
os.mkdir('dataset_1')
os.mkdir('dataset_2')

os.mkdir('dataset_1/train')
os.mkdir('dataset_1/valid')

os.mkdir('dataset_2/train')
os.mkdir('dataset_2/valid')

In [ ]:
for k, sub_class in enumerate(os.listdir('../input/100-bird-species/train/')):
    sub_path = os.path.join('../input/100-bird-species/train', sub_class)
    dst = os.path.join('dataset_1/train', sub_class)

    if not os.path.isdir(dst):
        os.mkdir(dst)

    for file in os.listdir(sub_path):
        file_path = os.path.join(sub_path, file)
        img_arr = load_img_to_array(file_path)
        copies = aggressive_cropping(img_arr, 3, [128, 128, 3], [224, 224], [32, 32])

        for i, copy in enumerate(copies):
            array_to_img(copy).save(os.path.join(dst, f'c_{i}_{file}'))

        array_to_img(resize(img_arr, (32, 32))).save(os.path.join(dst, file))

    print(k + 1, end='\r')

In [ ]:
for k, sub_class in enumerate(os.listdir('')):
    sub_path = os.path.join('', sub_class)
    dst = os.path.join('dataset_2/train', sub_class)

    if not os.path.isdir(dst):
        os.mkdir(dst)

    for file in os.listdir(sub_path):
        file_path = os.path.join(sub_path, file)
        img_arr = load_img_to_array(file_path)
        copies = aggressive_cropping(img_arr, 3, [128, 128, 3], [224, 224], [32, 32])

        for i, copy in enumerate(copies):
            array_to_img(copy).save(os.path.join(dst, f'c_{i}_{file}'))

        array_to_img(resize(img_arr, (32, 32))).save(os.path.join(dst, file))

    print(k + 1, end='\r')

In [ ]:
train_generator = ImageDataGenerator(rescale=1 / 255.,
                                     horizontal_flip=True)

valid_generator = ImageDataGenerator(rescale=1 / 255.,
                                     horizontal_flip=True)

In [ ]:
train_set = train_generator.flow_from_directory('../input/intel-image-classification/seg_train/seg_train/',
                                                target_size=(224, 224),
                                                batch_size=32)

val_set = valid_generator.flow_from_directory('../input/intel-image-classification/seg_test/seg_test/',
                                              target_size=(224, 224),
                                              batch_size=32)

train_set_1 = train_generator.flow_from_directory('../input/intel-image-classification/seg_train/seg_train/',
                                                target_size=(224, 224),
                                                batch_size=32)

val_set_1 = valid_generator.flow_from_directory('../input/intel-image-classification/seg_test/seg_test/',
                                              target_size=(224, 224),
                                              batch_size=32)

### ***Models***

In [ ]:
init = HeNormal()
classes = [6, 400]
act_name = ['relu','lrelu','elu']

In [ ]:
def scheduler(epoch,lr):
    if epoch<35000:
        return 0.01
    elif 35000<epoch<85000:
        return 0.005
    elif 85000<epoch<135000:
        return 0.0005
    elif 135000<epoch<165000:
        return 0.00005

lr_schedule = LearningRateScheduler(scheduler)

In [ ]:
activation_learn_history_0 = {}

for act in act_name:
    learning_history = []

    for _ in range(10):
        model1 = Sequential()

        model1.add(InputLayer((32, 32, 1)))
        model1.add(Flatten())

        for _ in range(8):
            model1.add(Dense(units=128, kernel_initializer=init))

            if act=='relu':
                model1.add(ReLU())
            elif act=='lrelu':
                model1.add(LeakyReLU(alpha=0.1))
            elif act=='elu':
                model1.add(ELU())

        model1.add(Dense(units=classes[0], activation='softmax'))

        model1.compile(optimizer=SGD(learning_rate=0.01),
                       loss=categorical_crossentropy,
                       metrics=['accuracy'])

        history = model1.fit(train_set_1,
                             validation_data=val_set_1,
                             steps_per_epoch=256,
                             validation_steps=128,
                             epochs=300,
                             batch_size=64)

        learning_history.append(history.history)

    activation_learn_history_0[act_name[i]] = learning_history

with open('result_0.json','w') as file:
    json.dump(activation_learn_history_0,file)

In [ ]:
activation_learn_history_1 = {}

for i in range(3):
    learning_history = []

    for _ in range(10):
        model2 = Sequential()

        model2.add(InputLayer((32, 32, 3)))
        model2.add(Conv2D(filters=192, kernel_size=(5, 5), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model2.add(Dropout(rate=0.0))

        model2.add(Conv2D(filters=192, kernel_size=(1, 1), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2.add(Conv2D(filters=240, kernel_size=(3, 3), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model2.add(Dropout(rate=0.1))

        model2.add(Conv2D(filters=240, kernel_size=(1, 1), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2.add(Conv2D(filters=260, kernel_size=(2, 2), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model2.add(Dropout(rate=0.2))

        model2.add(Conv2D(filters=260, kernel_size=(1, 1), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2.add(Conv2D(filters=280, kernel_size=(2, 2), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model2.add(Dropout(rate=0.3))

        model2.add(Conv2D(filters=280, kernel_size=(1, 1), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2.add(Conv2D(filters=300, kernel_size=(2, 2), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model2.add(Dropout(rate=0.4))

        model2.add(Conv2D(filters=300, kernel_size=(1, 1), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model2.add(Dropout(rate=0.5))

        model2.add(Flatten())
        model2.add(Dense(filters=classes[1], activation=softmax))

        model2.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9),
                       loss=categorical_crossentropy,
                       metrics=['accuracy'])

        history = model2.fit(train_set,
                             validation_data=val_set,
                             steps_per_epoch=256,
                             validation_steps=128,
                             epochs=165000,
                             batch_size=128,
                             callbacks=[lr_schedule])

        learning_history.append(history.history)

    activation_learn_history_1[act_name[i]] = learning_history

with open('result_1.json','w') as file:
    json.dump(activation_learn_history_1,file)

In [ ]:
activation_learn_history_2 = {}

for i in range(3):
    learning_history = []

    for _ in range(10):
        model2_bn = Sequential()

        model2_bn.add(InputLayer((32, 32, 3)))
        model2_bn.add(Conv2D(filters=192, kernel_size=(5, 5), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2_bn.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model2_bn.add(Dropout(rate=0.0))
        model2_bn.add(BatchNormalization())

        model2_bn.add(Conv2D(filters=192, kernel_size=(1, 1), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2_bn.add(Conv2D(filters=240, kernel_size=(3, 3), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2_bn.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model2_bn.add(Dropout(rate=0.1))
        model2_bn.add(BatchNormalization())

        model2_bn.add(Conv2D(filters=240, kernel_size=(1, 1), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2_bn.add(Conv2D(filters=260, kernel_size=(2, 2), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2_bn.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model2_bn.add(Dropout(rate=0.2))
        model2_bn.add(BatchNormalization())

        model2_bn.add(Conv2D(filters=260, kernel_size=(1, 1), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2_bn.add(Conv2D(filters=280, kernel_size=(2, 2), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2_bn.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model2_bn.add(Dropout(rate=0.3))
        model2_bn.add(BatchNormalization())

        model2_bn.add(Conv2D(filters=280, kernel_size=(1, 1), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2_bn.add(Conv2D(filters=300, kernel_size=(2, 2), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2_bn.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model2_bn.add(Dropout(rate=0.4))
        model2_bn.add(BatchNormalization())

        model2_bn.add(Conv2D(filters=300, kernel_size=(1, 1), kernel_initializer=init, activation=activations[i],
                          kernel_regularizer=l2(l2=0.0005)))
        model2_bn.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model2_bn.add(Dropout(rate=0.5))
        model2_bn.add(BatchNormalization())

        model2_bn.add(Flatten())
        model2_bn.add(Dense(filters=classes[1], activation=softmax))

        model2_bn.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9),
                       loss=categorical_crossentropy,
                       metrics=['accuracy'])

        history = model2_bn.fit(train_set,
                                validation_data=val_set,
                                steps_per_epoch=256,
                                validation_steps=128,
                                epochs=165000,
                                batch_size=128,
                                callbacks=[lr_schedule])

        learning_history.append(history.history)

    activation_learn_history_2[act_name[i]] = learning_history

with open('result_2.json','w') as file:
    json.dump(activation_learn_history_2,file)